In [2]:
%%time

from adlfs import AzureBlobFileSystem


import time



import json
import pandas
import pandas as pd
from tqdm import tqdm

from common.schemas.pyarrow_schema import tagging_schema
from common.storage.azure_file_storage import AzureFileStorageAdapter
from common.captioning.azure_descriptions import AzureCaption
from common.schemas.pyarrow_schema import schema
from common.functions.functions import Functions

tqdm.pandas(desc="Progress")
file_system: AzureBlobFileSystem = AzureFileStorageAdapter('data').get_file_storage()

functions: Functions = Functions()

caption: AzureCaption = AzureCaption(file_system)

CPU times: total: 4.12 s
Wall time: 37.1 s
CPU times: total: 0 ns
Wall time: 85.1 ms


In [3]:
%%time

curated_data = pandas.read_parquet("data/parquet/back.parquet", engine="pyarrow", filesystem=file_system)

curated_data.set_index("id", inplace=True, drop=False)

filtered = curated_data.loc[curated_data["accept"] == True, schema.names]

filtered.dropna(inplace=True)

display(filtered.shape)

display(filtered)

(19721, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
1013bdt,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
105mekt,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
105qvgl,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
105rpcj,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
105styc,105styc,AmIhotAF,Gizzygirl127,Low key… still bangable?,smiling woman sitting on couch with remote con...,6d555943be4fbc21ff92417c6f582298,/r/AmIhotAF/comments/105styc/low_key_still_ban...,https://i.redd.it/aiaxxoz9uoaa1.jpg,105styc.jpg,data/image/105styc.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13j46r0,13j46r0,DLAH,Optimus141,Tanned skin and white toes,araffe woman in a brown dress sitting on a whi...,03c478889ea9580f5dfb14e56448b96f,/r/DLAH/comments/13j46r0/tanned_skin_and_white...,https://i.redd.it/7ppevdffd80b1.jpg,13j46r0.jpg,data/image/13j46r0.jpg,PrettyGirlDiffusion,True,True,True,[]
13j4uk3,13j4uk3,DLAH,Thin_Data_2134,DLAHx4,three women in white dresses posing for a pict...,4ad5d6bd50df82a56a7981a390a73a0f,/r/DLAH/comments/13j4uk3/dlahx4/,https://i.redd.it/joexlbl9i80b1.jpg,13j4uk3.jpg,data/image/13j4uk3.jpg,PrettyGirlDiffusion,True,True,True,[]
13itj75,13itj75,bathandbodyworks,meg_mann,Small haul! It’s been quite a while since I’ve...,a close up of a bottle of body wash and a bott...,d37961f6bc630168919105e39c948611,/r/bathandbodyworks/comments/13itj75/small_hau...,https://i.redd.it/mbj4rd2rk50b1.jpg,13itj75.jpg,data/image/13itj75.jpg,CandleDiffusion,True,True,True,[]


CPU times: total: 328 ms
Wall time: 5.35 s


In [4]:
%%time

sources = [
	{"name": "CityDiffusion", "data": ["CityPorn"]},
	{"name": "NatureDiffusion", "data": ["EarthPorn"]},
	{"name": "CosmicDiffusion", "data": ["spaceporn"]},
	{"name": "ITAPDiffusion", "data": ["itookapicture"]},
	{"name": "MemeDiffusion", "data": ["memes"]},
	{"name": "TTTDiffusion", "data": ["trippinthroughtime"]},
	{"name": "WallStreetDiffusion", "data": ["wallstreetbets"]},
	{"name": "SexyDiffusion", "data": ["selfies", "Amicute", "amihot", "AmIhotAF", "HotGirlNextDoor"]},
	{"name": "FatSquirrelDiffusion", "data": ["fatsquirrelhate"]},
	{"name": "CelebrityDiffusion", "data": ["celebrities"]},
	{"name": "OldLadyDiffusion", "data": ["oldladiesbakingpies"]},
	{"name": "SWFPetite", "data": ["sfwpetite"]},
	{"name": "SFWMilfs", "data": ["cougars_and_milfs_sfw"]},
	{"name": "RedHeadDiffusion", "data": ["SFWRedheads"]},
	{"name": "NextDoorGirlsDiffusion", "data": ["SFWNextDoorGirls"]},
	{"name": "SexyAsianDiffusion","data": ["realasians", "KoreanHotties", "prettyasiangirls", "AsianOfficeLady", "AsianInvasion","AesPleasingAsianGirls"]},
	{"name": "MildlyPenisDiffusion", "data": ["mildlypenis"]},
	{"name": "PrettyGirlDiffusion","data": ["sexygirls", "PrettyGirls", "gentlemanboners", "hotofficegirls", "tightdresses", "DLAH"]},
	{"name": "CandleDiffusion", "data": ["bathandbodyworks"]}
]
sources_df = pd.DataFrame.from_records(sources)

CPU times: total: 0 ns
Wall time: 0 ns


In [5]:
filtered['model'] = filtered.apply(lambda x: functions.add_source(x, sources), axis=1)
foo = filtered.loc[filtered['model'] != ""]
foo.dropna()
filtered = foo
display(filtered)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
1013bdt,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
105mekt,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
105qvgl,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
105rpcj,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
105styc,105styc,AmIhotAF,Gizzygirl127,Low key… still bangable?,smiling woman sitting on couch with remote con...,6d555943be4fbc21ff92417c6f582298,/r/AmIhotAF/comments/105styc/low_key_still_ban...,https://i.redd.it/aiaxxoz9uoaa1.jpg,105styc.jpg,data/image/105styc.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13j46r0,13j46r0,DLAH,Optimus141,Tanned skin and white toes,araffe woman in a brown dress sitting on a whi...,03c478889ea9580f5dfb14e56448b96f,/r/DLAH/comments/13j46r0/tanned_skin_and_white...,https://i.redd.it/7ppevdffd80b1.jpg,13j46r0.jpg,data/image/13j46r0.jpg,PrettyGirlDiffusion,True,True,True,[]
13j4uk3,13j4uk3,DLAH,Thin_Data_2134,DLAHx4,three women in white dresses posing for a pict...,4ad5d6bd50df82a56a7981a390a73a0f,/r/DLAH/comments/13j4uk3/dlahx4/,https://i.redd.it/joexlbl9i80b1.jpg,13j4uk3.jpg,data/image/13j4uk3.jpg,PrettyGirlDiffusion,True,True,True,[]
13itj75,13itj75,bathandbodyworks,meg_mann,Small haul! It’s been quite a while since I’ve...,a close up of a bottle of body wash and a bott...,d37961f6bc630168919105e39c948611,/r/bathandbodyworks/comments/13itj75/small_hau...,https://i.redd.it/mbj4rd2rk50b1.jpg,13itj75.jpg,data/image/13itj75.jpg,CandleDiffusion,True,True,True,[]


In [6]:
%%time

group = filtered[["id", "subreddit"]].groupby("subreddit").count().sort_values(by="id", ascending=False)
plot = group.plot.bar(figsize=(20, 10), title="Subreddits with most posts", legend=True)
display(plot)

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

In [7]:
%%time

group = filtered[["id", "model", "subreddit"]].groupby(["model"]).count().sort_values(by="id", ascending=False)
plot_1 = group.plot.bar(figsize=(20, 10), title="Models with most images", legend=True)
display(plot_1)

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.

In [8]:
bar = filtered.loc[(filtered["model"] == "SexyDiffusion") | (filtered["model"] == "SexyAsianDiffusion") | (filtered['model'] == "NextDoorGirlDiffusion") | (filtered['model'] == 'RedHeadDiffusion')]
dropped = bar.dropna(inplace=True)
display(bar.shape)
display(bar)

C:\Users\ajsta\AppData\Local\Temp\ipykernel_2848\2056004052.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dropped = bar.dropna(inplace=True)


(2263, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
1013bdt,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
105mekt,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
105qvgl,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
105rpcj,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
105styc,105styc,AmIhotAF,Gizzygirl127,Low key… still bangable?,smiling woman sitting on couch with remote con...,6d555943be4fbc21ff92417c6f582298,/r/AmIhotAF/comments/105styc/low_key_still_ban...,https://i.redd.it/aiaxxoz9uoaa1.jpg,105styc.jpg,data/image/105styc.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13j2sd6,13j2sd6,realasians,lakdh,Anyone likes this cutie?,araffe woman in a skirt posing for a picture i...,3cfff5fbae7c3a153334156879c2233d,/r/realasians/comments/13j2sd6/anyone_likes_th...,https://i.redd.it/iifq9cie280b1.jpg,13j2sd6.jpg,data/image/13j2sd6.jpg,SexyAsianDiffusion,True,True,True,[]
13ivr6i,13ivr6i,prettyasiangirls,ReggieWarr,Booty Shorts and Boots,there is a woman leaning on a railing with a h...,44c3f9066e339742ccf949af35781de1,/r/prettyasiangirls/comments/13ivr6i/booty_sho...,https://i.redd.it/dyfyup7dn40b1.jpg,13ivr6i.jpg,data/image/13ivr6i.jpg,SexyAsianDiffusion,True,True,True,[]
13ixnk0,13ixnk0,prettyasiangirls,RightBachAtYa,Striped dress shirt and white underwear,arafed woman in a white underwear posing for a...,0bb02e71a351b8656ee59b552650aab6,/r/prettyasiangirls/comments/13ixnk0/striped_d...,https://i.redd.it/lx7a6n6i650b1.jpg,13ixnk0.jpg,data/image/13ixnk0.jpg,SexyAsianDiffusion,True,True,True,[]


In [ ]:
%%time

import datetime
from common.captioning.azure_descriptions import AzureCaption

records = bar.to_dict(orient="records")
current_captions = [item.replace('\n', '') for item in file_system.ls("data/caption")]

i = 0
start_time = datetime.datetime.now()
for elem in records:
	op_time = datetime.datetime.now()
	time_current = f"{(op_time - start_time).total_seconds()/60} minutes"
	i += 1
	path = elem['path']
	remote_path = file_system.url(path)

	if f'data/caption/{elem["id"]}.json' not in current_captions:
		display(f'Processing {elem["id"]} -- {i}/{len(records)}...{time_current}', clear=True)
		caption: AzureCaption = AzureCaption(file_system)
		output = caption.image_analysis(remote_path)
		time.sleep(12)
		json_result = output.json_result
		if json_result is None:
			display(f'Error with {elem["id"]} -- Empty Result -- {i}/{len(records)}...{time_current}', clear=True)
			continue
		try:
			if json.loads(json_result).get('error'):
				display(f'Error with {elem["id"]} -- Error: {json.loads(json_result).get("error")} -- {i}/{len(records)}...{time_current}', clear=True)
				continue
			handle = open('temp.json', 'w', encoding='utf-8')
			handle.write(json_result)
			handle.close()
			display(f'Uploading {elem["id"]} -- {i}/{len(records)}...{time_current}', clear=True)
			file_system.upload('temp.json', f'data/caption/{elem["id"]}.json')
		except Exception as e:
			display(f'Error with {elem["id"]} -- Error: {e} -- {i}/{len(records)}', clear=True)
			continue
	else:
		display(f'Skipping {elem["id"]} -- Already Processed -- {i}/{len(records)}...{time_current}', clear=True)
		continue

'Skipping yukvpy -- Already Processed -- 461/2263...0.14514651666666667 minutes'

In [ ]:
# !jupyter notebook stop